In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Reshape, Attention, TimeDistributed, Embedding, LSTM, Bidirectional, Dropout, Dense, GRU, concatenate, Conv1D, Conv2D, Flatten,MaxPooling1D, MaxPooling2D, LocallyConnected1D, Activation, GaussianNoise, BatchNormalization, RepeatVector
from tensorflow.keras.models import Model, Sequential
import nltk
from nltk.corpus import stopwords
from AttentionWeightedAverage import AttentionWeightedAverage
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

- NLP on Title + Description to Tags, in order to build a tag recommendation algorithm
- NLP on Tags + Description + Category to Title, in order to provide title recommendation algorithm. These videos are popular videos, meaning they have appealing titles. Let's build a system, that judging by content, can provide the most appropriate title.

In [2]:
df = pd.read_csv("./data/USvideos.csv")
print('Number of youtube videos in dataframe: ', len(df.index))
df.head()

# count category occurences
occurences = df['category_id'].value_counts()
# category names
map_categories = {}
with open('data/US_category_id.json') as json_file:
    data_category = json.load(json_file)
    for category in data_category['items']:
        int_id = int(category['id'])
        occ = 0
        if int_id in occurences:
            occ = occurences[int_id]
        map_categories[int_id] = [category['snippet']['title'],occ]
# create new dataframe with categories and appearances
df_categories = pd.DataFrame.from_dict(map_categories, orient='index', columns=['category_name', 'count'])
# add category name on the main dataframe
for i in map_categories.keys():
    df.loc[df['category_id'] == i,'category_name'] = map_categories[i][0]

Number of youtube videos in dataframe:  40949


### Tags, Description and Category -> Auto generate Title
We will see a word level seq2seq model

In [3]:
def discretize(tags):
    tags = tags.replace('"','')
    discrete_tags = tags.split('|')
    return discrete_tags

In [4]:
df_text = df[['title', 'description', 'tags', 'category_name']]
df_text = df_text.dropna(axis=0, how='any')

titles = df_text['title'].tolist()
descriptions = df_text['description'].tolist()
tags = [discretize(row['tags']) for index, row in df_text.iterrows()]
categories = df_text['category_name'].tolist()

- Titles need to be tokenized, lowercased, categorical.
- Descriptions need to be tokenized, lowercased, categorical.
- Tags need to be lowercased, categorical.

In [5]:
stops = set(stopwords.words("english"))

In [6]:
clean_desc = []
clean_title = []
for desc in descriptions:
    desc = desc.replace('\\n', ' ')
    desc = desc.lower()
    desc = desc.replace('www.facebook.com', 'FacebookLink')
    desc = desc.replace('http.facebook.com', 'FacebookLink')
    desc = desc.replace('www.twitter.com', 'TwitterLink')
    desc = desc.replace('www.amazon.com', 'AmazonLink')
    desc = desc.replace('http.amzn.com', 'AmazonLink')
    desc = desc.replace('www.instagram.com', 'InstagramLink')
    desc = desc.replace('www.snapchat.com', 'SnapchatLink')
    desc = desc.replace('https://bit.ly', 'PortalLink')
    temp = text_to_word_sequence(desc, filters='!"#%&()*-+,.\\/:;<=>?@[\\]^_`{|}~\t\n♡▶➜', lower=True, split=' ')
    for w in temp:
        if w in stops:
            temp.remove(w)
    clean_desc.append(temp)
    
for title in titles:
    title = title.replace('\\n', ' ')
    title = title.replace('\xa0', ' ')
    seq = text_to_word_sequence(title, filters='"#%&()*-+,.\\/:;<=>?@[\\]^_`{|}~\t\n♡▶➜', lower=True, split=' ')
    clean_title.append(['START_ '] + seq + [' _END'])

Training an unsupervised method for word2vec with gensim, in order to provide me with very good embeddings

In [7]:
from gensim.models import Word2Vec
input_features = 100

gensim_model = Word2Vec(size=input_features,
                       window=5,
                       min_count=1,
                       sg=1,
                       hs=0,
                       negative=10,
                       workers=4)
gensim_model.build_vocab(clean_desc+clean_title+tags)
gensim_model.train(clean_desc+clean_title+tags, total_examples=len(clean_desc+clean_title+tags), epochs=10)

(57578805, 64014370)

Compute the vocabulary for the words that appear in the titles

In [8]:
all_title_words=set()
for title in clean_title:
    for word in title:
        if word not in all_title_words:
            all_title_words.add(word)

The encoder and decoder tokens defer, the encoder takes words from the description, while the decoder produces words from the titles

In [9]:
num_encoder_tokens = len(gensim_model.wv.vocab)

target_words = sorted(list(all_title_words))
num_decoder_tokens = len(all_title_words)

Creating my new sequences with vector representations of words

In [10]:
X_train_description = []
X_train_category = []
X_train_tags = []

lb_cat = LabelBinarizer()
lb_cat.fit(list(set(categories)))

maxLenDesc = 0
for desc in clean_desc:
    X_train_description.append(np.asarray([gensim_model.wv[word] for word in desc]))
    if len(desc) > maxLenDesc:
        maxLenDesc = len(desc)
        
maxLenTags = 0
for taglist in tags:
    X_train_tags.append(np.asarray([gensim_model.wv[tag] for tag in taglist]))
    if len(taglist) > maxLenTags:
        maxLenTags = len(taglist)

for cat in categories:
    X_train_category.append(lb_cat.transform([cat]))

maxLenTitle = 0
for title in clean_title:
    if len(title) > maxLenTitle:
        maxLenTitle = len(title)

Create the decoder inputs and outputs. Decoder target data will be ahead by one timestep, and will not include the start character.

In [11]:
X_train_title = np.zeros(
    (len(clean_title), maxLenTitle, input_features),
    dtype='float32')

Y_train_title = np.zeros(
    (len(clean_title), maxLenTitle, num_decoder_tokens),
    dtype='float32')

# generate data
for i, target_text in enumerate(clean_title):
    for t, word in enumerate(target_text):
        # decoder input data, will have the embeddings of the words
        X_train_title[i, t] = gensim_model.wv[word]
        if t > 0:
            # decoder target data, will be ahead by one timestep, and will contain
            # one hot representations of what word is selected
            Y_train_title[i, t - 1, target_words.index(word)] = 1.

Pad Sequences

In [12]:
# padding
X_train_description = pad_sequences(X_train_description, maxlen=maxLenDesc, padding='post')
X_train_tags = pad_sequences(X_train_tags, maxlen=maxLenTags, padding='post')

# convert to numpy arrays
X_train_description = np.stack(X_train_description, axis=0)
X_train_category = np.stack(X_train_category, axis=0)
X_train_tags = np.stack(X_train_tags, axis=0)

Train - Test Split
* Encoder inputs
* Decoder inputs + outputs

In [13]:
# encoder + decoder inputs
X_train_description, X_test_description, X_train_title, X_test_title = train_test_split(X_train_description, X_train_title, test_size=0.05, shuffle=False)

In [14]:
num_titles = len(clean_title)
test_size = X_test_description.shape[0]
# decoder outputs
Y_train_title, Y_test_title = (Y_train_title[:num_titles-test_size], Y_train_title[num_titles-test_size:num_titles])

### Attention Layer

In [15]:
# https://github.com/thushv89/attention_keras/blob/master/layers/attention.py
# Attention Layer implemented by thushv89
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

### Sequence to Sequence model - seq2seq

In [16]:
def Seq2SeqAttentionModel(vocab_size,input_max_seq,output_target_seq,input_features):
    #encoder
    encoder_input = Input(shape=(input_max_seq,input_features), name='main_input')
    encoder_lstm = LSTM(100, return_sequences=True, return_state=True, name='encoder_lstm')
    encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_input)
    encoder_states = [encoder_state_h, encoder_state_c]
    
    
    #decoder
    decoder_input = Input(shape=(None,input_features)) 
    decoder_lstm = LSTM(100, return_sequences=True, return_state=True, name='decoder_lstm')
    # lstm
    decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_input, initial_state=encoder_states)
    
    # attention
    attention_layer = AttentionLayer(name='AttentionLayer')
    attention_out, attention_states = attention_layer([encoder_outputs,decoder_outputs])
    
    # concat
    concat = concatenate([decoder_outputs, attention_out], axis=-1, name='concat_layer')
    
    # dense
    decoder_dense = Dense(vocab_size, name='decoder_dense', activation='softmax')
    decoder_dense = TimeDistributed(decoder_dense, name='time_distrib_layer')
    decoder_outputs = decoder_dense(concat)
    
    # model inference
    model = Model([encoder_input, decoder_input], decoder_outputs)
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=4e-3))
    # encoder model
    encoder_model = Model(encoder_input, [encoder_outputs, encoder_states])

    # decoder inference
    decoder_init_states = [Input(shape=(100,)), Input(shape=(100,))]
    encoder_inf_outputs = Input(shape=(input_max_seq, input_features))
    decoder_inf_outputs, state_h, state_c = decoder_lstm(decoder_input, initial_state=decoder_init_states)
    decoder_inf_states = [state_h, state_c]
    # attn
    attn_inf_out, attn_inf_states = attention_layer([encoder_inf_outputs, decoder_inf_outputs])
    decoder_inf_concat = concatenate(inputs=[decoder_inf_outputs, attn_inf_out], axis=-1, name='concat')
    # dense
    decoder_inf_outputs = decoder_dense(decoder_inf_concat)
    decoder_model = Model([decoder_input, encoder_inf_outputs, decoder_init_states], [decoder_inf_outputs, attn_inf_states, decoder_inf_states])
    
    return model,encoder_model,decoder_model

In [19]:
epochs=1
batch_size=64
model,encoder_model,decoder_model = Seq2SeqAttentionModel(num_decoder_tokens,maxLenDesc,maxLenTitle,100)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 722, 100)]   0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 722, 100), ( 80400       main_input[0][0]                 
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 100),  80400       input_5[0][0]                    
                                                                 encoder_lstm[0][1]         

In [67]:
model.fit([X_train_description, X_train_title], Y_train_title, epochs=epochs, batch_size=batch_size,verbose=1)

Train on 38360 samples
38360/38360 [==============================] - 2934s 76ms/sample - loss: 0.4821


In [68]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    enc_pred = encoder_model.predict(input_seq)
    encoder_outputs, h, c = enc_pred[0], enc_pred[1], enc_pred[2]
    # Generate empty target sequence of length 1.
    target_seq = gensim_model.wv['START_ ']
    target_seq = target_seq.reshape((1,1,target_seq.shape[0]))
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, attention, h, c = decoder_model.predict([target_seq, encoder_outputs, h, c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_words[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == ' _END' or len(decoded_sentence) > maxLenTitle):
            stop_condition = True

        target_seq = np.append(target_seq, gensim_model.wv[sampled_word].reshape((1,1,input_features)), axis=1)
        # Update states
        states_value = [h, c]

    return decoded_sentence

In [69]:
i = 0
for i in list(range(20)):
    title_generated = decode_sequence(X_test_description[i].reshape((1,X_test_description.shape[1],X_test_description.shape[2])))
    print("i: ", i)
    print("Description:\n",descriptions[-test_size+i])
    print("\nGenerated Title: ", title_generated.replace(' _END', ''))
    print("\n\n")
    i+=1

i:  0
Description:
 Thanks for an amazing year!(Im pretty behind on my next video.....)SomeThingElseYT: https://www.youtube.com/c/SomeThingElseYT

Generated Title:   22 years it real life 



i:  1
Description:
 Many musicians prefer these 300-year-old instruments, but are they actually worth it?Subscribe to our channel! http://goo.gl/0bsAjOAntonio Stradivari is generally considered the greatest violin maker of all time. His violins are played by some of the top musicians in the world and sell for as much as $16 million. For centuries people have puzzled over what makes his violins so great and they are the most scientifically studied instruments in history. I spoke to two world class violinists who play Stradivarius violins as well as a violin-maker about what makes Stradivari so great. Special thanks to Stefan Avalos for the Stradivari research footage.Vox.com is a news website that helps you cut through the noise and understand what's really driving the events in the headlines. Chec

i:  6
Description:
 With a busy schedule, Jocko Willink finds time to get everything done by waking up before everyone else does. Willink, former Navy SEAL and author of Way of the Warrior Kid explains the one habit from service that he can't shake.For the full interview, search for Success! How I Did It on Apple Podcasts or your favorite app. https://itunes.apple.com/us/podcast/success-how-i-did-it/id1205997729?mt=2 Business Insider tells you all you need to know about business, finance, tech, science, retail, and more.Subscribe to our channel and visit us at: http://www.businessinsider.com/BI on Facebook: https://www.facebook.com/businessinsider/BI on Instagram: https://www.instagram.com/businessinsider/BI on Twitter: https://twitter.com/businessinsider--------------------------------------------------Following is a transcript of the video:Richard Feloni: Are there some things from your service that you can't shake? So for example, you still wake up at 4:30 in the morning, to go work

i:  10
Description:
 MEANING OF LIFE available now: https://Atlantic.lnk.to/MeaningoflifeIDShop “I Don’t Think About You” Merch: http://smarturl.it/IDTAYMerchFollow Kelly Clarkson: https://kellyclarkson.com/https://www.facebook.com/kellyclarksonhttps://twitter.com/kelly_clarksonhttps://www.instagram.com/kellyclarkson/

Generated Title:   kelly clarkson meaning



i:  11
Description:
 NanoDots master builder, Edo, turned a huge pile of 25,000 NanoDots magnets into the Infinity Gauntlet from Avengers: Infinity War.Buy here and learn how to build cool Nanodots structures: https://www.vat19.com/item/nanodots-spherical-neodymium-magnets?adid=youtubeSubscribe to Vat19: http://www.youtube.com/subscription_center?add_user=vat19comEdo's channel: https://www.youtube.com/user/EdoTimmermans****************** Follow Vat19:Facebook: https://facebook.com/vat19Instagram: https://instagram.com/vat19/Twitter: https://twitter.com/vat19Shop hundreds more curiously awesome products:https://www.vat19.com/?a